<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 0.Imports

In [43]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")
print('finished')

finished


## <span style='color:#ff5f27'> 🌍 1.1 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [44]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()

2024-12-12 20:15:37,404 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-12 20:15:37,423 INFO: Initializing external client
2024-12-12 20:15:37,423 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-12 20:15:39,138 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1150100


In [45]:


fs = project.get_feature_store() 
#secrets = util.secrets_api(project.name)
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
#Now we just need to modify the first nb. The other ones are automatically updated
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

print(location_str)

{"country": "italy", "city": "toscana", "street": "pisa", "aqicn_url": "https://api.waqi.info/feed/@9432", "latitude": 43.47, "longitude": 10.68}


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [46]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
weather_fg

---

## <span style='color:#ff5f27'> 2.Retrieve Datas using APIs</span>


### <span style='color:#ff5f27'> 2.1 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [47]:
#import requests
#import pandas as pd
#yesterday = today - datetime.timedelta(days=3)

#aq_tmp_df = util.get_pm25(aqicn_url, country, city, street, yesterday, AQI_API_KEY)
# Function that gives back df from API request 
#aq_tmp_df

In [48]:
#fin_df = pd.concat([fin_df,aq_tmp_df])

#fin_df

In [49]:
#air_quality_fg.insert(existing_df)

In [50]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,30.0,italy,toscana,pisa,2024-12-12,https://api.waqi.info/feed/@9432


In [51]:
#aq_today_df.info()

In [52]:
dfa = air_quality_fg.read()
dfa = dfa.sort_values(by='date', ascending=False)  # Sorts in ascending order
dfa

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.03s) 


,date,pm25,country,city,street,url
2764,2024-12-12 00:00:00+00:00,30.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2763,2024-11-25 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2762,2024-11-24 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2761,2024-11-23 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2760,2024-11-22 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
...,...,...,...,...,...,...
1403,2017-01-07 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
1998,2017-01-06 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
1331,2017-01-05 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
730,2017-01-04 00:00:00+00:00,82.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432


### <span style='color:#ff5f27'> 2.2 Get Weather Forecast data</span>

In [53]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
# Function that gives back df from API request 
hourly_df.head()

Coordinates 43.5°N 10.75°E
Elevation 147.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2024-12-12 00:00:00,2.5,0.0,4.024922,116.564987
1,2024-12-12 01:00:00,2.0,0.0,4.198285,120.963692
2,2024-12-12 02:00:00,1.4,0.0,4.693826,122.471176
3,2024-12-12 03:00:00,1.0,0.0,4.896530,126.027466
4,2024-12-12 04:00:00,0.6,0.0,5.191994,123.690094


In [54]:
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df.head()

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-12-12,10.70,0.0,9.449572,72.255241,toscana
1,2024-12-13,7.40,0.0,6.849467,86.987282,toscana
2,2024-12-14,9.15,0.3,4.693825,184.398621,toscana
3,2024-12-15,9.95,0.0,4.104631,74.744827,toscana
4,2024-12-16,11.55,0.0,0.804985,296.564972,toscana


In [55]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">3.3 ⬆️ Uploading new data to the Feature Store</span>

In [56]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-12-12 20:15:50,722 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1354080


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1150100/jobs/named/air_quality_2_offline_fg_materialization/executions


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677934
         }
       },
       "result": {
         "observed_value": 30.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-12T07:15:50.000722Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [57]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-12-12 20:16:02,639 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1354080


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677934
         }
       },
       "result": {
         "observed_value": 30.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-12T07:16:02.000639Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [58]:
# Insert new data
weather_fg.insert(daily_df)

2024-12-12 20:16:10,327 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1345889


Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1150100/jobs/named/weather_1_offline_fg_materialization/executions


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669711
         }
       },
       "result": {
         "observed_value": 0.8049845099449158,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-12T07:16:10.000326Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type"

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [59]:
dfa['pm25_rolling'] = dfa['pm25'].rolling(window=3, min_periods=1).mean()
dfa['pm25_rolling'] = dfa['pm25_rolling'].fillna(dfa['pm25'])
dfa

,date,pm25,country,city,street,url,pm25_rolling
2764,2024-12-12 00:00:00+00:00,30.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,30.000000
2763,2024-11-25 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,40.000000
2762,2024-11-24 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,40.666667
2761,2024-11-23 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,44.666667
2760,2024-11-22 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,42.000000
...,...,...,...,...,...,...,...
1403,2017-01-07 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,81.000000
1998,2017-01-06 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,57.666667
1331,2017-01-05 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,49.000000
730,2017-01-04 00:00:00+00:00,82.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,58.000000


## Adding the Rolling Windows Datagroup

In [60]:
air_quality_roll_fg = fs.get_or_create_feature_group(
    name='air_quality_roll',
    description='Air Quality characteristics of each day, rolling feature',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date"
)

In [61]:
air_quality_roll_fg.insert(dfa)

Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████| Rows 2765/2765 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_roll_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1150100/jobs/named/air_quality_roll_1_offline_fg_materialization/executions


(Job('air_quality_roll_1_offline_fg_materialization', 'SPARK'), None)

In [62]:
df_lagged = air_quality_roll_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.10s) 


In [63]:
df_lagged = df_lagged.sort_values(by='date', ascending=False)
df_lagged

,date,pm25,country,city,street,url,pm25_rolling
2764,2024-12-12 00:00:00+00:00,30.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,30.000000
2763,2024-11-25 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,40.000000
2762,2024-11-24 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,40.666667
2761,2024-11-23 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,44.666667
2760,2024-11-22 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,42.000000
...,...,...,...,...,...,...,...
1984,2017-01-07 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,81.000000
377,2017-01-06 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,57.666667
946,2017-01-05 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,49.000000
728,2017-01-04 00:00:00+00:00,82.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,58.000000
